# 04_build_scenes Prompt API Playground

Scope: API-only prompt testing for `harness/prompts/04_build_scenes`.

This notebook is self-contained and does **not** invoke the pipeline or write scene files.

## Cell 1: Setup

Purpose:
- Resolve repo imports from notebook context.
- Load `.env` automatically.
- Import prompt template helpers and API client.

In [1]:
import json
import os
import re
import sys
from pathlib import Path

# Resolve repo root whether cwd is repo root, notebooks/, or another child folder.
cwd = Path.cwd().resolve()
candidates = [cwd, *cwd.parents]
repo_root = None
for p in candidates:
    if (p / 'harness').exists() and (p / 'notebooks').exists():
        repo_root = p
        break
if repo_root is None:
    raise RuntimeError('Could not locate repo root containing harness/ and notebooks/')
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

# Load environment from repo .env if possible.
env_path = repo_root / '.env'
try:
    from dotenv import load_dotenv
    if env_path.exists():
        load_dotenv(env_path)
except Exception:
    pass

from harness.prompts import load_prompt_template, read_file, PROMPTS_DIR, TEMPLATES_DIR
from harness.client import LLMClient, estimate_tokens

print(f'Repo root: {repo_root}')

Repo root: /Users/velocityworks/IdeaProjects/flaming-horse


## Cell 2: Hardcoded Build Inputs

Purpose:
- Define `plan_data`, `narration_script`, and scene state inline.
- Select which scene to build via `current_scene_index`.

In [2]:
plan_data = {
    "title": "How Matrix Multiplication Works",
    "description": "A step-by-step guide to understanding matrix multiplication.",
    "target_duration_seconds": 360,
    "scenes": [
        {
            "id": "scene_01_intro",
            "title": "Introduction to Matrix Multiplication",
            "narration_key": "scene_01_intro",
            "description": "Introduce motivation and preview structure.",
            "estimated_duration_seconds": 25,
            "visual_ideas": [
                "Title and matrix icons",
                "A and B slide in",
                "C = A * B appears"
            ]
        },
        {
            "id": "scene_02_matrix_basics",
            "title": "What is a Matrix?",
            "narration_key": "scene_02_matrix_basics",
            "description": "Define matrix, rows/columns, notation.",
            "estimated_duration_seconds": 30,
            "visual_ideas": [
                "Grid fills with values",
                "Row/column labels",
                "Vector vs matrix comparison"
            ]
        }
    ]
}

narration_script = {
    "scene_01_intro": "Welcome to how matrix multiplication works. We'll see why it matters, then walk through the core rule and examples.",
    "scene_02_matrix_basics": "A matrix is a rectangular grid of numbers with rows and columns. We'll use dimensions and index notation to describe each element clearly."
}

def scene_id_to_class_name(scene_id: str) -> str:
    parts = [p for p in str(scene_id).split('_') if p]
    return ''.join(p.capitalize() for p in parts)

# Mirrors the state layout harness expects for build_scenes.
state = {
    "scenes": [
        {
            "id": "scene_01_intro",
            "title": "Introduction to Matrix Multiplication",
            "file": "scene_01_intro.py",
            "class_name": "Scene01Intro",
            "narration_key": "scene_01_intro"
        },
        {
            "id": "scene_02_matrix_basics",
            "title": "What is a Matrix?",
            "file": "scene_02_matrix_basics.py",
            "class_name": "Scene02MatrixBasics",
            "narration_key": "scene_02_matrix_basics"
        }
    ],
    "current_scene_index": 0
}

# Optional retry details for scene regeneration tests.
retry_context = None

temperature = 0.7
max_tokens = 16000

## Cell 3: Compose Prompts

Purpose:
- Compose `build_scenes` system/user prompts using exact harness templates.
- Fill placeholders from the in-notebook state/plan/narration data.

In [3]:
scenes = state.get('scenes', [])
current_index = state.get('current_scene_index', 0)
assert 0 <= current_index < len(scenes), 'current_scene_index out of range'

current_scene = scenes[current_index]
scene_id = current_scene.get('id', f'scene_{current_index + 1:02d}')
scene_title = current_scene.get('title', 'Unknown')
narration_key = current_scene.get('narration_key', scene_id)
scene_file_name = current_scene.get('file', f'{scene_id}.py')
scene_class_name = current_scene.get('class_name', scene_id_to_class_name(scene_id))

plan_scene = next((s for s in plan_data.get('scenes', []) if s.get('id') == scene_id), None)
scene_details = json.dumps(plan_scene, indent=2) if plan_scene else 'N/A'

scene_narration = narration_script.get(narration_key)
if not scene_narration:
    raise ValueError(f'Missing narration for key: {narration_key}')

reference_section = ''
if current_index == 0:
    reference_section = (
        '## Manim CE Reference Documentation\n\n'
        'Use ONLY syntax documented in the official Manim CE reference:\n'
        'https://docs.manim.community/en/stable/reference.html\n\n'
        'This is your authoritative source for valid Manim classes, methods, and parameters.'
    )

retry_section = ''
if retry_context:
    retry_section = (
        '## RETRY CONTEXT\n\n'
        'This scene previously failed with the following error:\n\n'
        '```\n'
        f'{retry_context}\n'
        '```\n\n'
        'Please fix the issue and generate a corrected version.'
    )

system_prompt = load_prompt_template(
    'build_scenes',
    'system.md',
    {
        'core_rules': read_file(PROMPTS_DIR / '_shared' / 'core_rules.md'),
        'config_guide': read_file(TEMPLATES_DIR / 'manim_config_guide.md'),
        'visual_helpers': read_file(TEMPLATES_DIR / 'visual_helpers.md'),
        'kitchen_sink': read_file(TEMPLATES_DIR / 'kitchen_sink.md'),
    },
)

user_prompt = load_prompt_template(
    'build_scenes',
    'user.md',
    {
        'scene_id': scene_id,
        'scene_file_name': scene_file_name,
        'scene_class_name': scene_class_name,
        'narration_key': narration_key,
        'scene_title': scene_title,
        'scene_details': scene_details,
        'scene_narration': scene_narration,
        'reference_section': reference_section,
        'retry_section': retry_section,
    },
)

print(f'Current scene: {scene_id} ({scene_title})')
print(f'System prompt chars: {len(system_prompt):,}')
print(f'User prompt chars:   {len(user_prompt):,}')
print(f'System est tokens:   {estimate_tokens(system_prompt):,}')
print(f'User est tokens:     {estimate_tokens(user_prompt):,}')

Current scene: scene_01_intro (Introduction to Matrix Multiplication)
System prompt chars: 42,592
User prompt chars:   3,417
System est tokens:   10,648
User est tokens:     854


## Cell 4: Prompt Preview

Purpose:
- Inspect prompt snippets before sending to API.

In [4]:
print('=== SYSTEM PROMPT PREVIEW ===')
print(system_prompt[:2000])
print('\n=== USER PROMPT PREVIEW ===')
print(user_prompt[:2500])

=== SYSTEM PROMPT PREVIEW ===
Video Production Agent - Build Scenes Phase

# Core Rules for Flaming Horse Agent

**These rules apply to ALL phases and MUST NEVER be violated.**

---

## 🚨 CRITICAL: VOICE POLICY - READ THIS FIRST

### Local Qwen Voice Clone - No Fallback

**ABSOLUTE REQUIREMENTS:**
- ✅ **ONLY** local Qwen voice clone audio cached on disk (no network TTS).
- ✅ **Model:** `Qwen/Qwen3-TTS-12Hz-1.7B-Base` (voice clone).
- ✅ **Device/Dtype:** CPU `float32` for stability.
- ✅ **Reference assets:** `assets/voice_ref/ref.wav` + `assets/voice_ref/ref.txt` per project.
- ❌ **NEVER** use any network TTS service (e.g., ElevenLabs, Google TTS, Azure TTS).
- ❌ **NEVER** create fallback code patterns.

**If cached audio is missing, the build MUST fail and instruct to run the precache step.**

---

## 🚨 CRITICAL: Execution Protocol

**When the user says "proceed", "execute", "continue", or "approve":**

- ❌ **NEVER** present another detailed plan
- ❌ **NEVER** ask "Does this align with

## Cell 5: API Call

Purpose:
- Send composed prompts to the LLM API and print raw response text.

In [5]:
provider = os.getenv('LLM_PROVIDER', 'XAI').upper()
api_key_var = f'{provider}_API_KEY'
assert os.getenv(api_key_var), f'Missing {api_key_var} in environment/.env'

client = LLMClient()
response_text = client.chat_completion(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    temperature=temperature,
    max_tokens=max_tokens,
)

print(response_text[:7000])

🤖 Harness using:
   Provider: XAI
   Base URL: https://api.x.ai/v1
   Model: grok-4-1-fast-reasoning
<scene_body>
    # Title
    title = Text("Introduction to Matrix Multiplication", font_size=48, weight=BOLD)
    title.move_to(UP * 3.8)
    title.set_color(BLUE)

    self.play(polished_fade_in(title))

    # Matrix A (initially offscreen left)
    matrix_A = MathTex(
        r"\begin{bmatrix} 1 & 2 \\ 3 & 4 \end{bmatrix}",
        font_size=48
    )
    matrix_A.set_color(BLUE)
    matrix_A.move_to(LEFT * 3.5 + DOWN * 0.6)

    label_A = Text("A", font_size=36, weight=BOLD)
    label_A.set_color(BLUE)
    label_A.next_to(matrix_A, DOWN, buff=0.3)
    safe_position(label_A)

    group_A = VGroup(matrix_A, label_A)
    group_A.shift(LEFT * 3)

    # Matrix B (initially offscreen right)
    matrix_B = MathTex(
        r"\begin{bmatrix} 5 & 6 \\ 7 & 8 \end{bmatrix}",
        font_size=48
    )
    matrix_B.set_color(GREEN)
    matrix_B.move_to(RIGHT * 3.5 + DOWN * 0.6)

    label_B = Tex

## Cell 6: Response Sanity Checks

Purpose:
- Run lightweight checks on returned code text.
- No files are written.

In [6]:
checks = {
    'contains class name': scene_class_name in response_text,
    'contains VoiceoverScene': 'VoiceoverScene' in response_text,
    'contains narration key': narration_key in response_text,
    'imports get_speech_service': 'get_speech_service' in response_text,
}

for k, v in checks.items():
    print(f'{k}: {v}')

missing = [k for k, v in checks.items() if not v]
print('missing checks:', missing)

contains class name: False
contains VoiceoverScene: False
contains narration key: True
imports get_speech_service: False
missing checks: ['contains class name', 'contains VoiceoverScene', 'imports get_speech_service']
